Lesson 3

---

# 🧑‍🍳 LangChain: Prompt Template

**Outline**

* Direct API calls to OpenAI
  
* API calls through LangChain:
  * Models  
  * Prompts


🔴 <font color="red">Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the class.</font>

## ⚙️ Setup 

[OpenAI API Key](https://platform.openai.com/account/api-keys)

In [3]:
import re
from util import local_settings
from openai import OpenAI

from env_colors import TerminalTextColor

model="gpt-3.5-turbo"

print("First LLM API example")
print(f"✅ OpenAI Key loaded (...{local_settings.OPENAI_API_KEY[20:-20]}...)")
print(f"✅ Model: {model}")


First LLM API example
✅ OpenAI Key loaded (...QopT3BlbkFJ...)
✅ Model: gpt-3.5-turbo


> 🔔 <font color="#00d4d4">**Note:** some characters of the key are omitted for security reasons.</font>

In [ ]:
client = OpenAI(api_key=local_settings.OPENAI_API_KEY)

def get_completion(prompt, temperature= 0, messages = [], model=model):

    message = {"role": "user", "content": prompt}

    messages.append(message)

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    return completion.choices[0].message.content

In [ ]:
messages = [
    {
        "system" : "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know."
    }
]

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

```mermaid
flowchart LR
    Email --> Format
    Style_1 --> Format
    Prompt_template --> Format
    Format --> Prompt
    Prompt --> LLM
    LLM --> Response
```
The chain view

### Customer e-mail

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of
cleaning up me kitchen. I need yer help right now, matey!
"""

### Style

In [ ]:
style = """American English in a calm and respectful tone"""

### Prompt

In [ ]:
prompt = """Translate the text that is delimited by triple backtick into a style that is {style}. In the end, add a portuguese translation of the response.

text: ```{customer_email}```"""

print(prompt)

In [ ]:
print(re.findall(r'{(.*?)}', prompt))

In [ ]:
prompt.format( style=style, customer_email=customer_email )

In [ ]:
prompt_inputs = { "style":style, "customer_email":customer_email }
prompt.format( **prompt_inputs )

In [ ]:
prompt = f"""Translate the text that is delimited by triple backtick into a style that is {style}.
In the end, add a portuguese translation of the response.
text: ```{customer_email}```"""

print(f"{TerminalTextColor.BLUE}----- PROMPT ------{TerminalTextColor.RESET}")
print(f"{TerminalTextColor.GREEN}{prompt}{TerminalTextColor.RESET}")

In [ ]:
response = get_completion(prompt)
print(f"{TerminalTextColor.BLUE}----- RESPONSE ------{TerminalTextColor.RESET}")
print(f"{TerminalTextColor.YELLOW}{response}{TerminalTextColor.RESET}")

## Chat API : LangChain

Let's try how we can do the same using LangChain.

```mermaid
flowchart LR
    Email --> ChatPromptTemplate_1
    Style_1 --> ChatPromptTemplate_1
    Prompt_template_1 --> ChatPromptTemplate_1
    ChatPromptTemplate_1 --> Prompt_1
    Prompt_1 --> LLM_1"Chat"
    LLM_1"Chat" --> Result_1
    Service_response --> ChatPromptTemplate_2
    Result_1 --> ChatPromptTemplate_2
    Style_2 --> ChatPromptTemplate_2
    Prompt_template_2 --> ChatPromptTemplate_2
    ChatPromptTemplate_2 --> Prompt_2
    Prompt_2 --> LLM_2
    LLM_2 --> Final_Response
```

The chain view


In [ ]:
#

https://mermaid.js.org/syntax/flowchart.html

### Model

In [ ]:
from langchain.chat_models import ChatOpenAI
model="gpt-3.5-turbo"

In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=model)

print(f"{TerminalTextColor.BLUE}----- chat ------{TerminalTextColor.RESET}")
print(chat)

### Prompt template

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
template_string = """Translate the text that is delimited by triple backticks \
into a style that is {style}. In the end, add a portuguese translation of the response.

text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)


print(f"{TerminalTextColor.BLUE}----- prompt_template ------{TerminalTextColor.RESET}")
print(prompt_template)

In [ ]:
print(f"{TerminalTextColor.BLUE}----- ...messages[0].prompt ------{TerminalTextColor.RESET}")

prompt_template.messages[0].prompt

In [ ]:
print(f"{TerminalTextColor.BLUE}----- input_variables ------{TerminalTextColor.RESET}")
prompt_template.messages[0].prompt.input_variables

### Style and E-mail

In [ ]:
customer_style = """American English in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
"""

In [ ]:
get_completion(prompt=f"translate the following text to portuguese. TEXT: ```{customer_email}```")

In [ ]:
customer_messages = prompt_template.format_messages(
    style=customer_style, text=customer_email
)
print(f"{TerminalTextColor.BLUE}----- type of customer_messages ------{TerminalTextColor.RESET}")
print(type(customer_messages))

print(f"{TerminalTextColor.BLUE}----- type of customer_messages[0] ------{TerminalTextColor.RESET}")
print(type(customer_messages[0]))

print(f"{TerminalTextColor.BLUE}----- customer_messages[0] ------{TerminalTextColor.RESET}")
print(customer_messages[0])

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)
print(f"{TerminalTextColor.BLUE}----- RESPONSE ------{TerminalTextColor.RESET}")

print(customer_response.content)

In [ ]:
service_reply = """Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender.

Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """a polite tone that speaks in English Pirate"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

<h3><font color="Yellow">👋 the end </font></h3>